In [1]:
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
from datetime import datetime, time
import glob
from autogluon.tabular import TabularDataset, TabularPredictor
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from tqdm import tqdm
from meteostat import Daily

In [2]:
def display_errors(y_true, y_pred, return_vals=False):
    mse = mean_squared_error(y_true, y_pred)
    print(f"MSE: {round(mse, 3)}")
    rmse = mse ** 0.5
    print(f"RMSE: {round(rmse, 3)}")
    mae = mean_absolute_error(y_true, y_pred)
    print(f"MAE: {round(mae, 3)}")
    mape = mean_absolute_percentage_error(y_true, y_pred)
    print(f"MAPE: {round(mape, 3)}")
    r2 = r2_score(y_true, y_pred)
    print(f"R2: {round(r2, 3)}")

    if return_vals:
        return [mse, rmse, mae, mape, r2]

def plot_errors(y_true, y_pred, name='N/A'):
    plt.scatter(y_true, y_pred, color='blue', marker='o', label=f'error adjusted predictions ({name}) vs actual degree days')
    plt.xlabel('actual degree days')
    plt.ylabel('predicted degree days')
    plt.title(f'Prediction vs Actual ({name})')

    max_value = max(max(y_true[-225:]), max(y_true[-225:]))
    min_value = min(min(y_true[-225:]), min(y_true[-225:]))
    plt.plot([min_value, max_value], [min_value, max_value], color='red', linestyle='--', label='Line of Equality')

    plt.legend()

In [3]:
class LinReg:
    def __init__(self, train_data, test_data, target_col):
        self.X_train = train_data.drop(columns=[target_col])
        self.y_train = train_data[target_col]
        self.X_test = test_data.drop(columns=[target_col])
        self.y_test = test_data[target_col]
        self.target_col = target_col
        self.model = LinearRegression().fit(self.X_train, self.y_train)
        self.y_pred = self.model.predict(self.X_test)
        self.evaluate()

    def evaluate(self):
        display_errors(self.y_test, self.y_pred)


In [4]:
def extract_date_time(filename):
    """
    extract the date and time from the filename
    :param filename:
    :return:
    """
    parts = filename.split('.')
    extracted_date = parts[1]
    extracted_time = parts[2]
    return extracted_date, extracted_time

def get_date(file):
        """get the date from the dataframe and the time from the filename and combine them into a datetime object
        :param file: filename containing the time
        :return: datetime object
        """

        date_str = str(file.split('.')[1])
        time_str = str(file.split('.')[2])
        date_value = datetime.strptime(date_str, '%Y%m%d')
        time_value = time(int(time_str), 0)
        combined_datetime = datetime.combine(date_value.date(), time_value)
        return combined_datetime
#give type hint on start_date that it should be a string
def seasonal_train_test(data, start_date: str, end_date: str):
    """
    split the data into train and test sets based on dates
    :param data: dataframe containing the data
    :param start_date: start date of the test set
    :param end_date: end date of the test set
    :return: train and test sets
    """

    test_start = start_date
    test_end = end_date
    test_data = data.loc[test_start: test_end]

    #remove X_test from dataframe to get X_train
    mask = (data.index >= test_start) & (data.index <= test_end)
    train_data = data.loc[~mask]

    return train_data, test_data

In [27]:
class RawDataProcess:
    def __init__(self, degree_days='gw_hdd', path='RawData', time=None):
        self.degree_days = degree_days
        self.path = path
        self.sort_files()
        self.get_master_model()
        if time is not None:
            self.filter_time(time)


    def filter_time(self, time):
        data = self.master_data
        data['hour'] = data.index.hour
        data = data[data['hour'] == time]
        data = data.drop('hour', axis=1)
        self.master_data = data

    def sort_files(self):
        """
        sort the files in the directory by date and time
        :return:
        """
        degree_days = self.degree_days
        path = self.path

        degree_days = degree_days
        ecmwf_files = glob.glob(path + f'/ecmwf.*.[01][02].{degree_days}.csv')
        ecmwf_sorted_files = sorted(ecmwf_files, key=lambda x: (x.split('.')[1], x.split('.')[2]))[3:]

        ecmwf_ens_files = glob.glob(path + f'/ecmwf-eps.*.[01][02].{degree_days}.csv')
        ecmwf_ens_sorted_files = sorted(ecmwf_ens_files, key=lambda x: (x.split('.')[1], x.split('.')[2]))[2:]

        gfs_ens_bc_files = glob.glob(path + f'/gfs-ens-bc.*.[01][02].{degree_days}.csv')
        gfs_ens_bc_sorted_files = sorted(gfs_ens_bc_files, key=lambda x: (x.split('.')[1], x.split('.')[2]))[2:]

        cmc_ens_files = glob.glob(path + f'/cmc-ens.*.[01][02].{degree_days}.csv')
        cmc_ens_sorted_files = sorted(cmc_ens_files, key=lambda x: (x.split('.')[1], x.split('.')[2]))[2:]
        for _ in range(2):
            set1 = set((extract_date_time(filename) for filename in ecmwf_sorted_files))
            set2 = set((extract_date_time(filename) for filename in ecmwf_ens_sorted_files))

            ecmwf_sorted_files = [filename for filename in ecmwf_sorted_files if extract_date_time(filename) in set2]
            ecmwf_ens_sorted_files = [filename for filename in ecmwf_ens_sorted_files if
                                      extract_date_time(filename) in set1]
            cmc_ens_sorted_files = [filename for filename in cmc_ens_sorted_files if extract_date_time(filename) in set1]

            master_set = set((extract_date_time(filename) for filename in cmc_ens_sorted_files))
            gfs_ens_bc_sorted_files = [filename for filename in gfs_ens_bc_sorted_files if
                                       extract_date_time(filename) in master_set]

            master_set = set((extract_date_time(filename) for filename in gfs_ens_bc_sorted_files))

            ecmwf_sorted_files = [filename for filename in ecmwf_sorted_files if extract_date_time(filename) in master_set]
            ecmwf_ens_sorted_files = [filename for filename in ecmwf_ens_sorted_files if
                                      extract_date_time(filename) in master_set]
            gfs_ens_bc_sorted_files = [filename for filename in gfs_ens_bc_sorted_files if
                                       extract_date_time(filename) in master_set]
            cmc_ens_sorted_files = [filename for filename in cmc_ens_sorted_files if
                                    extract_date_time(filename) in master_set]

        self.ecmwf_sorted_files = ecmwf_sorted_files
        self.ecmwf_ens_sorted_files = ecmwf_ens_sorted_files
        self.gfs_ens_bc_sorted_files = gfs_ens_bc_sorted_files
        self.cmc_ens_sorted_files = cmc_ens_sorted_files

    def y_value(self, start=8, end=14):
        ecmwf_ens_9_14 = pd.DataFrame(columns=[f'ens({start+1},{end})'])

        ecmwf_ens_sorted_files = self.ecmwf_ens_sorted_files

        for i in range(1, len(ecmwf_ens_sorted_files)):
            ecmwf_ens_df = pd.read_csv(ecmwf_ens_sorted_files[i])
            ecmwf_ens_df = ecmwf_ens_df[ecmwf_ens_df[ecmwf_ens_df.columns[2]] >= 1]
            prev_ecmwf_ens_df = pd.read_csv(ecmwf_ens_sorted_files[i - 1])
            prev_ecmwf_ens_df = prev_ecmwf_ens_df[prev_ecmwf_ens_df[prev_ecmwf_ens_df.columns[2]] >= 1]

            date = get_date(ecmwf_ens_sorted_files[i])
            prev_date = get_date(ecmwf_ens_sorted_files[i - 1])
            d2 = str(date)[:10]
            d1 = str(prev_date)[:10]

            if d2 == d1:
                offset = 1
            else:
                offset = 0

            cur = ecmwf_ens_df['Value'].iloc[start:end].sum()
            prev = prev_ecmwf_ens_df['Value'].iloc[(start+offset):(end+offset)].sum()
            change = cur - prev

            new_row = pd.DataFrame(change, columns=ecmwf_ens_9_14.columns, index=[date])
            ecmwf_ens_9_14 = pd.concat([ecmwf_ens_9_14, new_row])

        self.y_values = ecmwf_ens_9_14

    def ecmwf_ens(self, start=7, end=8):
        ecmwf_ens_8 = pd.DataFrame(columns=[f'ens({end})'])
        ecmwf_ens_sorted_files = self.ecmwf_ens_sorted_files

        for i in range(1, len(ecmwf_ens_sorted_files)):
            ecmwf_ens_df = pd.read_csv(ecmwf_ens_sorted_files[i])
            ecmwf_ens_df = ecmwf_ens_df[ecmwf_ens_df[ecmwf_ens_df.columns[2]] >= 1]
            prev_ecmwf_ens_df = pd.read_csv(ecmwf_ens_sorted_files[i - 1])
            prev_ecmwf_ens_df = prev_ecmwf_ens_df[prev_ecmwf_ens_df[prev_ecmwf_ens_df.columns[2]] >= 1]

            date = get_date(ecmwf_ens_sorted_files[i])
            prev_date = get_date(ecmwf_ens_sorted_files[i - 1])
            d2 = str(date)[:10]
            d1 = str(prev_date)[:10]

            if d2 == d1:
                offset = 1
            else:
                offset = 0

            cur = ecmwf_ens_df['Value'].iloc[start:end].sum() #7-8 benchmark, 7-8 best results
            prev = prev_ecmwf_ens_df['Value'].iloc[(start+offset):(end+offset)].sum() #7-8 benchmark, 7-8 best results
            change = cur - prev

            new_row = pd.DataFrame(change, columns=ecmwf_ens_8.columns, index=[date])
            ecmwf_ens_8 = pd.concat([ecmwf_ens_8, new_row])

        self.ecmwf_ens_data = ecmwf_ens_8

    def ecmwf(self, start=8, end=9):

        ecmwf_9_10 = pd.DataFrame(columns=[f'ecmwf({end})'])

        ecmwf_sorted_files = self.ecmwf_sorted_files
        ecmwf_ens_sorted_files = self.ecmwf_ens_sorted_files


        for i in range(1, len(ecmwf_sorted_files)):
            ecmwf_df = pd.read_csv(ecmwf_sorted_files[i])
            ecmwf_df = ecmwf_df[ecmwf_df[ecmwf_df.columns[2]] >= 1]
            prev_ecmwf_ens_df = pd.read_csv(ecmwf_ens_sorted_files[i-1])
            prev_ecmwf_ens_df = prev_ecmwf_ens_df[prev_ecmwf_ens_df[prev_ecmwf_ens_df.columns[2]] >= 1]

            date = get_date(ecmwf_sorted_files[i])
            prev_date = get_date(ecmwf_sorted_files[i - 1])
            d2 = str(date)[:10]
            d1 = str(prev_date)[:10]

            if d2 == d1:
                offset = 1
            else:
                offset = 0

            cur = ecmwf_df['Value'].iloc[start:end].sum() #8-9 benchmark, 4-9 best results
            prev = prev_ecmwf_ens_df['Value'].iloc[(start+offset):(end+offset)].sum() #8-9 benchmark, 4-9 best results

            change = cur - prev

            new_row = pd.DataFrame(change, columns=ecmwf_9_10.columns, index=[date])
            ecmwf_9_10 = pd.concat([ecmwf_9_10, new_row])

        self.ecmwf_data = ecmwf_9_10

    def gfs(self, start=9, end=14):
        gfs_11_14 = pd.DataFrame(columns=[f'gfs({start+1},{end})'])

        ecmwf_ens_sorted_files = self.ecmwf_ens_sorted_files
        gfs_ens_bc_sorted_files = self.gfs_ens_bc_sorted_files

        for i in range(1, len(gfs_ens_bc_sorted_files)):
            gfs_df = pd.read_csv(gfs_ens_bc_sorted_files[i])
            gfs_df = gfs_df[gfs_df[gfs_df.columns[2]] >= 1]
            prev_ecmwf_ens_df = pd.read_csv(ecmwf_ens_sorted_files[i-1])
            prev_ecmwf_ens_df = prev_ecmwf_ens_df[prev_ecmwf_ens_df[prev_ecmwf_ens_df.columns[2]] >= 1]

            date = get_date(gfs_ens_bc_sorted_files[i])
            prev_date = get_date(ecmwf_ens_sorted_files[i - 1])
            d2 = str(date)[:10]
            d1 = str(prev_date)[:10]

            if d2 == d1:
                offset = 1
            else:
                offset = 0

            cur = gfs_df['Value'].iloc[start:end].sum() # 9-14 benchmark, 9-16 best results
            prev = prev_ecmwf_ens_df['Value'].iloc[(start+offset):(end+offset)].sum() # 9-14 benchmark, 9-16 best results

            change = cur - prev

            new_row = pd.DataFrame(change, columns=gfs_11_14.columns, index=[date])
            gfs_11_14 = pd.concat([gfs_11_14, new_row])

        self.gfs_data = gfs_11_14

    def cmc(self, start=8, end=14):
        cmc_9_14 = pd.DataFrame(columns=[f'cmc({start+1},{end})'])

        cmc_ens_sorted_files = self.cmc_ens_sorted_files
        gfs_ens_bc_sorted_files = self.gfs_ens_bc_sorted_files

        for i in range(1, len(cmc_ens_sorted_files)):
            cmc_df = pd.read_csv(cmc_ens_sorted_files[i])
            cmc_df = cmc_df[cmc_df[cmc_df.columns[2]] >= 1]
            gfs_df = pd.read_csv(gfs_ens_bc_sorted_files[i])
            gfs_df = gfs_df[gfs_df[gfs_df.columns[2]] >= 1]

            date = get_date(cmc_ens_sorted_files[i])

            cmc = cmc_df['Value'].iloc[start:end].sum() #8-14 benchmark, 8-14 best results
            gfs = gfs_df['Value'].iloc[start:end].sum() #8-14 benchmark, 8-14 best results
            change = cmc - gfs

            new_row = pd.DataFrame(change, columns=cmc_9_14.columns, index=[date])
            cmc_9_14 = pd.concat([cmc_9_14, new_row])

        self.cmc_data = cmc_9_14

    def norm(self):
        norms = pd.DataFrame(columns=['Date', 'Value'])
        ecmwf_ens_sorted_files = self.ecmwf_ens_sorted_files

        for i in range(1, len(ecmwf_ens_sorted_files), 2):
            ecmwf_ens_df = pd.read_csv(ecmwf_ens_sorted_files[i])
            v1 = ecmwf_ens_df[ecmwf_ens_df[ecmwf_ens_df.columns[2]] == 2].iloc[:, :2]
            norms = pd.concat([norms, v1]).drop_duplicates('Date')

        norms.reset_index(inplace=True)
        norms.drop(columns=['index'], inplace=True)
        norms['Date'] = pd.to_datetime(norms['Date']).dt.strftime('%Y-%m-%d 12:00:00')
        norms.set_index('Date', inplace=True)
        norms.rename_axis('', inplace=True)
        norms.rename(columns={'Value': 'norm'}, inplace=True)

        self.norms_data = norms

    def run_all_models(self):
        self.y_value()
        self.ecmwf_ens()
        self.ecmwf()
        self.gfs()
        self.cmc()
        self.norm()

    def get_master_model(self):
        self.run_all_models()
        master_data = pd.concat([self.ecmwf_ens_data, self.ecmwf_data,
                                 self.gfs_data, self.cmc_data, self.y_values], axis=1)

        self.master_data = master_data

In [28]:
data = RawDataProcess(degree_days='gw_hdd', path='RawData', time=12)

In [9]:
master_data = data.master_data

In [12]:
class LoadWeatherData:
    def __init__(self, start=datetime(2018, 7, 11), end=datetime(2023, 6, 16)):
        self.start = start
        self.end = end
        self.load_stations()

    def load_stations(self, path='StormVistaData/station_df.csv'):
        stations = pd.read_csv(path)
        stations.rename(columns={'Unnamed: 0': 'id'}, inplace=True)
        self.stations = stations['id'].tolist()

    def process_weather_data(self, limited_cities=True):

        if limited_cities:
            citys = ['72278', '72295', '72565', '72508', 'H3GG0',
                     '72405', '72206', 'KFTY0', '72681', '72530', '72438',
                     '72546', '72451', '72435', '74754', '72606', '72406',
                     '72509', '72635', '72644', '72235', '72446', '72773']
        else:
            citys = self.stations

        weather_df = pd.DataFrame()
        for i in tqdm(citys):
            weather_data = Daily(i, self.start, self.end)
            weather_data = weather_data.fetch()
            weather_data = weather_data[['tavg', 'tmin', 'tmax', 'prcp', 'pres']]
            weather_data = weather_data.add_suffix(f'_{i}')
            weather_df = pd.concat([weather_df, weather_data], axis=1)
        weather_df.reset_index(inplace=True)
        weather_df['Date'] = pd.to_datetime(weather_df['time']).dt.strftime('%Y-%m-%d 12:00:00')
        weather_df.drop(columns=['time'], inplace=True)
        weather_df.set_index('Date', inplace=True)
        weather_df.rename_axis('', inplace=True)
        weather_df.index = pd.to_datetime(weather_df.index)

        return weather_df

In [13]:
weather_data = LoadWeatherData()
weather_df = weather_data.process_weather_data()

100%|██████████| 23/23 [00:17<00:00,  1.35it/s]


In [14]:
common_dates = master_data.index.intersection(weather_df.index)
master_data = pd.concat([master_data.loc[common_dates], weather_df.loc[common_dates]], axis=1)

In [15]:
display(master_data.head())

,ens(8),ecmwf(9),"gfs(10,14)","cmc(9,14)","ens(9,14)",tavg_72278,tmin_72278,tmax_72278,prcp_72278,pres_72278,...,tavg_72446,tmin_72446,tmax_72446,prcp_72446,pres_72446,tavg_72773,tmin_72773,tmax_72773,prcp_72773,pres_72773
2018-07-11 12:00:00,0.003,0.024,0.039,-0.040,0.024,30.4,26.1,38.3,0.0,NaN,...,30.3,24.4,37.2,0.0,1016.2,19.2,8.9,27.8,0.0,1018.6
2018-07-12 12:00:00,-0.002,0.005,0.046,-0.053,0.023,30.7,25.6,38.9,0.0,1010.9,...,30.6,24.4,37.8,0.0,1016.2,20.1,9.4,31.1,0.0,1017.7
2018-07-13 12:00:00,0.000,0.003,0.037,-0.048,0.029,34.1,30.0,41.1,0.0,1010.8,...,30.3,22.8,37.2,0.0,1015.4,22.1,11.1,33.3,0.0,1016.3
2018-07-14 12:00:00,-0.001,-0.001,0.046,-0.053,0.027,34.6,30.6,39.4,0.0,1010.1,...,26.8,22.2,31.7,0.5,1015.1,23.2,12.2,32.8,0.0,1015.7
2018-07-15 12:00:00,0.000,0.006,0.034,-0.039,0.033,33.2,31.1,37.8,0.0,1010.1,...,27.3,20.6,34.4,0.0,1014.5,22.8,12.8,32.2,0.0,1016.9


In [16]:
train_data, test_data = seasonal_train_test(master_data, '2022-10-04 12:00:00', '2023-5-16 12:00:00')

In [17]:
train_data = TabularDataset(train_data)
test_data = TabularDataset(test_data)

In [18]:
label = 'ens(9,14)'

In [19]:
y_test = test_data[label]
test_data_nolabel = test_data.drop(columns=[label])

In [20]:
predictor = TabularPredictor(label=label).fit(train_data) # presets='best_quality'

No path specified. Models will be saved in: "AutogluonModels/ag-20230712_203952/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20230712_203952/"
AutoGluon Version:  0.7.0
Python Version:     3.10.9
Operating System:   Darwin
Platform Machine:   x86_64
Platform Version:   Darwin Kernel Version 22.5.0: Thu Jun  8 22:22:19 PDT 2023; root:xnu-8796.121.3~7/RELEASE_ARM64_T8103
Train Data Rows:    1524
Train Data Columns: 119
Label Column: ens(9,14)
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (228.63299999999998, 0.0, 70.03885, 68.93051)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoML

In [21]:
fi = predictor.feature_importance(test_data, silent=True)

In [22]:
display(fi.head(20))

,importance,stddev,p_value,n,p99_high,p99_low
tavg_72278,3.112485,0.477146,0.000064,5,4.094935,2.130034
tmax_72681,2.442742,0.266307,0.000017,5,2.991071,1.894412
tmax_72773,2.199818,0.280980,0.000031,5,2.778360,1.621276
tmax_72278,1.548230,0.150555,0.000011,5,1.858226,1.238235
tavg_72681,1.124985,0.155303,0.000042,5,1.444757,0.805214
pres_72278,1.116491,0.272670,0.000395,5,1.677922,0.555061
tmin_72278,1.103012,0.185832,0.000093,5,1.485642,0.720383
tavg_72644,0.917157,0.119303,0.000034,5,1.162804,0.671510
tavg_72773,0.621653,0.101464,0.000082,5,0.830568,0.412737
tavg_72206,0.615980,0.195559,0.001071,5,1.018638,0.213322


In [23]:
y_pred = predictor.predict(test_data_nolabel)
display_errors(y_test, y_pred)

MSE: 1507.386
RMSE: 38.825
MAE: 29.193
MAPE: 0.239
R2: 0.498


In [24]:
#train errors
display_errors(train_data[label], predictor.predict(train_data.drop(columns=[label])))

MSE: 80.855
RMSE: 8.992
MAE: 5.612
MAPE: 6741567304427.611
R2: 0.983


In [25]:
linreg = LinReg(train_data, test_data, label)

ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

# weather data

In [19]:
stations = pd.read_csv('StormVistaData/station_df.csv')

In [20]:
stations.rename(columns={'Unnamed: 0': 'id'}, inplace=True)

In [21]:
station_ids = stations['id'].tolist()

In [22]:
start = datetime(2018, 7, 11)
end = datetime(2023, 6, 16)

In [23]:
display(stations[stations['region']=='MT'])

,id,icao,latitude,longitude,elevation,name,region,timezone
53,72772,KHLN,46.6000,-111.9667,1180.0,Helena Regional Airport,MT,America/Denver
68,72768,KGGW,48.2167,-106.6167,699.0,Glasgow International Airport,MT,America/Denver
168,72779,KGPI,48.3000,-114.2667,906.0,Glacier Park International Airport,MT,America/Denver
210,72775,KGTF,47.4667,-111.3833,1120.0,Great Falls International,MT,America/Denver
226,72777,KHVR,48.5500,-109.7667,789.0,Havre City-County Airport,MT,America/Denver
234,72677,KBIL,45.8167,-108.5500,1112.0,Logan International Airport,MT,America/Denver
245,74230,KMLS,46.4333,-105.8833,801.0,Miles City,MT,America/Denver
295,KBZN0,KBZN,45.7776,-111.1520,1363.0,Bozeman / Belgrade Village Mobile Home Park,MT,America/Denver
491,72773,KMSO,46.9167,-114.1000,976.0,Missoula International Airport,MT,America/Denver


In [24]:
citys = ['72278', '72295', '72565', '72508', 'H3GG0',
         '72405', '72206', 'KFTY0', '72681', '72530', '72438',
         '72546', '72451', '72435', '74754', '72606', '72406',
         '72509', '72635', '72644', '72235', '72446', '72773']

In [71]:
wd = Daily('72278', start, end).fetch()
display(wd)

,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
time,,,,,,,,,,
2018-07-11,30.4,26.1,38.3,0.0,0.0,141.0,13.7,NaN,NaN,NaN
2018-07-12,30.7,25.6,38.9,0.0,0.0,NaN,9.7,NaN,1010.9,NaN
2018-07-13,34.1,30.0,41.1,0.0,0.0,93.0,12.6,NaN,1010.8,NaN
2018-07-14,34.6,30.6,39.4,0.0,0.0,NaN,13.3,NaN,1010.1,NaN
2018-07-15,33.2,31.1,37.8,0.0,0.0,NaN,13.7,NaN,1010.1,NaN
...,...,...,...,...,...,...,...,...,...,...
2023-06-12,27.7,21.7,33.9,0.0,0.0,131.0,11.2,NaN,1009.4,NaN
2023-06-13,28.9,23.3,35.6,0.0,0.0,288.0,9.7,NaN,1009.2,NaN
2023-06-14,30.4,23.3,38.3,0.0,0.0,254.0,11.2,NaN,1005.9,NaN


In [55]:
# Get daily data
weather_df = pd.DataFrame()

for i in tqdm(citys):
    weather_data = Daily(i, start, end)
    weather_data = weather_data.fetch()
    weather_data = weather_data[['tavg', 'tmin', 'tmax', 'prcp', 'pres']]
    weather_data = weather_data.add_suffix(f'_{i}')
    weather_df = pd.concat([weather_df, weather_data], axis=1)

100%|██████████| 23/23 [00:00<00:00, 113.99it/s]


In [56]:
display(weather_df)

,tavg_72278,tmin_72278,tmax_72278,prcp_72278,pres_72278,tavg_72295,tmin_72295,tmax_72295,prcp_72295,pres_72295,...,tavg_72446,tmin_72446,tmax_72446,prcp_72446,pres_72446,tavg_72773,tmin_72773,tmax_72773,prcp_72773,pres_72773
time,,,,,,,,,,,,,,,,,,,,,
2018-07-11,30.4,26.1,38.3,0.0,NaN,23.8,21.7,26.7,0.0,1012.1,...,30.3,24.4,37.2,0.0,1016.2,19.2,8.9,27.8,0.0,1018.6
2018-07-12,30.7,25.6,38.9,0.0,1010.9,23.3,21.1,26.1,0.0,1013.8,...,30.6,24.4,37.8,0.0,1016.2,20.1,9.4,31.1,0.0,1017.7
2018-07-13,34.1,30.0,41.1,0.0,1010.8,22.2,20.6,24.4,0.0,1015.6,...,30.3,22.8,37.2,0.0,1015.4,22.1,11.1,33.3,0.0,1016.3
2018-07-14,34.6,30.6,39.4,0.0,1010.1,22.1,20.0,25.6,0.0,1014.5,...,26.8,22.2,31.7,0.5,1015.1,23.2,12.2,32.8,0.0,1015.7
2018-07-15,33.2,31.1,37.8,0.0,1010.1,22.7,20.6,25.6,0.0,1013.6,...,27.3,20.6,34.4,0.0,1014.5,22.8,12.8,32.2,0.0,1016.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-12,27.7,21.7,33.9,0.0,1009.4,17.5,15.6,21.1,0.0,1016.6,...,17.6,10.6,23.3,0.0,1014.4,18.1,14.4,27.2,3.6,1014.0
2023-06-13,28.9,23.3,35.6,0.0,1009.2,17.4,15.6,20.6,0.0,1016.4,...,19.1,10.0,26.7,0.0,1010.8,19.9,11.7,30.0,1.5,1010.3
2023-06-14,30.4,23.3,38.3,0.0,1005.9,16.6,15.0,18.3,0.0,1013.9,...,21.9,13.3,29.4,0.0,1006.4,18.7,12.2,21.7,0.0,1009.2


In [57]:
weather_df.reset_index(inplace=True)
weather_df['Date'] = pd.to_datetime(weather_df['time']).dt.strftime('%Y-%m-%d 12:00:00')
weather_df.drop(columns=['time'], inplace=True)
weather_df.set_index('Date', inplace=True)
weather_df.rename_axis('', inplace=True)
weather_df.index = pd.to_datetime(weather_df.index)

In [58]:
common_dates = master_data.index.intersection(weather_df.index)
master_data = pd.concat([master_data.loc[common_dates], weather_df.loc[common_dates]], axis=1)

# ------------------

In [59]:
display(master_data)

,ens(8),ecmwf(9),"gfs(10,14)","cmc(9,14)","ens(9,14)",tavg_72278,tmin_72278,tmax_72278,prcp_72278,pres_72278,...,tavg_72446,tmin_72446,tmax_72446,prcp_72446,pres_72446,tavg_72773,tmin_72773,tmax_72773,prcp_72773,pres_72773
2018-07-11 12:00:00,0.003,0.024,0.039,-0.040,0.024,30.4,26.1,38.3,0.0,NaN,...,30.3,24.4,37.2,0.0,1016.2,19.2,8.9,27.8,0.0,1018.6
2018-07-12 12:00:00,-0.002,0.005,0.046,-0.053,0.023,30.7,25.6,38.9,0.0,1010.9,...,30.6,24.4,37.8,0.0,1016.2,20.1,9.4,31.1,0.0,1017.7
2018-07-13 12:00:00,0.000,0.003,0.037,-0.048,0.029,34.1,30.0,41.1,0.0,1010.8,...,30.3,22.8,37.2,0.0,1015.4,22.1,11.1,33.3,0.0,1016.3
2018-07-14 12:00:00,-0.001,-0.001,0.046,-0.053,0.027,34.6,30.6,39.4,0.0,1010.1,...,26.8,22.2,31.7,0.5,1015.1,23.2,12.2,32.8,0.0,1015.7
2018-07-15 12:00:00,0.000,0.006,0.034,-0.039,0.033,33.2,31.1,37.8,0.0,1010.1,...,27.3,20.6,34.4,0.0,1014.5,22.8,12.8,32.2,0.0,1016.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-12 12:00:00,-1.098,-1.852,0.549,-2.337,14.607,27.0,18.9,36.1,0.0,1006.9,...,21.9,18.9,27.8,0.0,1010.6,13.6,4.4,24.4,0.0,1025.1
2023-05-13 12:00:00,-0.563,-0.920,1.340,0.595,10.157,29.3,21.7,37.8,0.0,1005.1,...,24.1,18.3,28.3,0.0,1014.5,16.0,6.1,22.2,0.0,1029.3
2023-05-14 12:00:00,0.210,-0.419,1.168,-2.417,10.311,30.8,22.8,38.3,0.0,1009.4,...,23.7,16.7,30.6,3.8,1019.9,17.9,12.2,23.9,0.0,1026.5
2023-05-15 12:00:00,-0.080,1.089,1.291,-2.618,9.604,32.2,25.6,39.4,0.0,1012.6,...,17.7,13.9,20.0,22.1,1022.3,20.2,12.2,28.9,0.0,1020.7
